In [1]:
# Just for the notebooks, the cwd needs to be set to the root of the project
import os
from pathlib import Path
home = Path.home()

cwd = os.getcwd()
if not 'initial_cwd' in locals():
	initial_cwd = cwd

# check if any of the parent directories is 'notebooks'
relative_path = Path(cwd).relative_to(home)
if 'notebooks' in relative_path.parts:
	# if so, change the current working directory to the root of the project
	while relative_path.parts and not 'notebooks' in os.listdir(cwd):
		cwd = os.path.dirname(cwd)
		relative_path = Path(cwd).relative_to(home)
	if 'notebooks' in os.listdir(cwd):
		os.chdir(cwd)

print(f"Current working directory: {os.getcwd()}")
print(f"Initial working directory: {initial_cwd}")

Current working directory: /home/cyphix/repos/pixel-ota
Initial working directory: /home/cyphix/repos/pixel-ota/notebooks


In [2]:
%pip install -r requirements.txt --quiet
from dotenv import load_dotenv
load_dotenv()


Note: you may need to restart the kernel to use updated packages.


True

# Setup logging

In [3]:
import logging
from LoggingColor import ColorHandler
import sys
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(name)-16s [%(levelname)-1s]: %(message)s',
    handlers=[
        logging.FileHandler(f'log.log',mode='w'),
        #logging.StreamHandler(sys.stdout),
        ColorHandler(sys.stdout)
    ],
    datefmt='%H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.debug("Debug log test")
logger.info("Info log test")
logger.warning("Warning log test")
logger.error("Error log test")
logger.critical("Critical log test")

19:58:29.649 __main__         [D]: Debug log test
19:58:29.649 __main__         [I]: Info log test
19:58:29.649 __main__         [W]: Warning log test
19:58:29.649 __main__         [E]: Error log test
19:58:29.649 __main__         [C]: Critical log test


In [4]:
import zipfile

download_dir = "./downloads"
avbroot_path = os.path.join(download_dir, "avbroot-3.22.0-x86_64-unknown-linux-gnu.zip")

# decompress avbroot so that it can be used
with zipfile.ZipFile(avbroot_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(download_dir, "avbroot"))

assert os.path.exists(os.path.join(download_dir, "avbroot")) and os.path.isdir(os.path.join(download_dir, "avbroot"))
assert os.path.exists(os.path.join(download_dir, "avbroot", "avbroot")) and os.path.isfile(os.path.join(download_dir, "avbroot", "avbroot"))
avbroot_path = os.path.join(download_dir, "avbroot", "avbroot")

import subprocess
if not os.access(avbroot_path, os.X_OK):
    # make it executable
    subprocess.run(["chmod", "+x", avbroot_path], check=True)
if not os.access(avbroot_path, os.X_OK):
    raise Exception(f"avbroot is not executable: {avbroot_path}")

KEY_AVB = "keys/avb.key"
PASSPHRASE_AVB = os.getenv("PASSPHRASE_AVB")
assert PASSPHRASE_AVB is not None, "PASSPHRASE_AVB environment variable is not set"
KEY_OTA = "keys/ota.key"
PASSPHRASE_OTA = os.getenv("PASSPHRASE_OTA")
assert PASSPHRASE_OTA is not None, "PASSPHRASE_OTA environment variable is not set"
CERT_OTA = "keys/ota.crt"

